In [81]:
!pip install evaluate
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments
import evaluate
import torch
import pandas as pd
import numpy as np
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [6]:
import pandas as pd
# Read csv and target the text for preprocessing
df_train = pd.read_csv("trac2_CONVT_train.csv")
columnsTrain = df_train["text"]

In [7]:

listSentence = []


for column in columnsTrain:
    listSentence.append(column) # Use original text


In [8]:
batch = tokenizer(listSentence, padding=True, truncation=True, return_tensors="pt")
print(batch["input_ids"])
print(batch["attention_mask"])

tensor([[ 101, 2054, 2106,  ...,    0,    0,    0],
        [ 101, 2009, 1005,  ...,    0,    0,    0],
        [ 101, 1045, 2228,  ...,    0,    0,    0],
        ...,
        [ 101, 2030, 2002,  ...,    0,    0,    0],
        [ 101, 1045, 2123,  ...,    0,    0,    0],
        [ 101, 7929, 4067,  ...,    0,    0,    0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])


In [9]:
labels_class = torch.tensor(df_train["EmotionalPolarity"].values)
labels_intensity = torch.tensor(df_train["Emotion"].values, dtype=torch.float)
labels_empathy = torch.tensor(df_train["Empathy"].values, dtype=torch.float)



In [10]:
from datasets import Dataset

hf_dataset = Dataset.from_dict({
    "input_ids": batch["input_ids"],
    "attention_mask": batch["attention_mask"],
    "labels_class": labels_class,
    "labels_intensity": labels_intensity,
    "labels_empathy": labels_empathy
})

In [11]:
hf_dataset = hf_dataset.train_test_split(test_size=0.2)
train_dataset = hf_dataset["train"]
val_dataset = hf_dataset["test"]

In [12]:
from transformers import BertModel, BertPreTrainedModel
import torch
from torch import nn

class BertMultiTask(BertPreTrainedModel):
    def __init__(self, config, num_labels=3):
        super().__init__(config)
        self.bert = BertModel(config)
        hidden = config.hidden_size
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(hidden, num_labels)  # classification head
        self.reg1 = nn.Linear(hidden, 1)                 # intensity head
        self.reg2 = nn.Linear(hidden, 1)                 # empathy head
        self.init_weights()

    def forward(self, input_ids, attention_mask=None,
                labels_class=None, labels_intensity=None, labels_empathy=None):
        out = self.bert(input_ids, attention_mask=attention_mask)
        pooled = self.dropout(out.pooler_output)
        logits = self.classifier(pooled)
        r1 = self.reg1(pooled).squeeze(-1)
        r2 = self.reg2(pooled).squeeze(-1)

        loss = None
        if labels_class is not None:
            loss_fct_cls = nn.CrossEntropyLoss()
            loss_fct_reg = nn.MSELoss()
            loss = (
                loss_fct_cls(logits, labels_class)
                + loss_fct_reg(r1, labels_intensity)
                + loss_fct_reg(r2, labels_empathy)
            )
        return {"loss": loss, "logits_cls": logits, "reg1": r1, "reg2": r2}


In [141]:
from transformers import BertModel, BertPreTrainedModel
import torch
from torch import nn

class BertMultiTask(BertPreTrainedModel):
    def __init__(self, config, num_labels=3):
        super().__init__(config)
        self.bert = BertModel(config)
        hidden = config.hidden_size
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(hidden, num_labels)
        self.reg1 = nn.Linear(hidden, 1)
        self.reg2 = nn.Linear(hidden, 1)
        self.init_weights()

    def forward(self, input_ids, attention_mask=None,
                labels_class=None, labels_intensity=None, labels_empathy=None):
        out = self.bert(input_ids, attention_mask=attention_mask)
        pooled = self.dropout(out.pooler_output)

        logits = self.classifier(pooled)
        probs_cls = torch.softmax(logits, dim=-1)

        r1 = self.reg1(pooled).squeeze(-1)
        r2 = self.reg2(pooled).squeeze(-1)

        loss = None
        if labels_class is not None:
            loss_fct_cls = nn.CrossEntropyLoss()
            loss_fct_reg = nn.MSELoss()
            loss = (
                loss_fct_cls(logits, labels_class)
                + loss_fct_reg(r1, labels_intensity)
                + loss_fct_reg(r2, labels_empathy)
            )
        return {"loss": loss, "logits_cls": logits, "reg1": r1, "reg2": r2}


In [142]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained("bert-base-uncased")
model = BertMultiTask.from_pretrained("bert-base-uncased", config=config, num_labels=3)


Some weights of BertMultiTask were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'reg1.bias', 'reg1.weight', 'reg2.bias', 'reg2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [143]:
import numpy as np
import evaluate

acc_metric = evaluate.load("accuracy")
mse_metric = evaluate.load("mse")


def compute_metrics(eval_pred):

    logits_cls, r1, r2 = eval_pred.predictions

    labels_cls, labels_r1, labels_r2 = eval_pred.label_ids

    preds_cls = np.argmax(logits_cls, axis=1)

    acc  = acc_metric.compute(predictions=preds_cls, references=labels_cls)
    mse1 = mse_metric.compute(predictions=r1, references=labels_r1)
    mse2 = mse_metric.compute(predictions=r2, references=labels_r2)

    return {
        "accuracy": acc["accuracy"],
        "mse_intensity": mse1["mse"],
        "mse_empathy": mse2["mse"]
    }

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [144]:
import transformers
print(transformers.__version__)

4.57.1


In [147]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./bert-multitask-results",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=4,
    weight_decay=0.01,

    eval_strategy="steps",
    save_strategy="epoch",
    eval_steps=50,
    logging_dir="./logs",
)


In [148]:
from transformers import Trainer
from transformers import TrainingArguments
from transformers import EarlyStoppingCallback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


/tmp/ipython-input-1203300166.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [149]:
trainer.train()


Step,Training Loss,Validation Loss,Accuracy,Mse Intensity,Mse Empathy
50,No log,2.273523,0.620202,0.424522,0.925451
100,No log,2.068879,0.657576,0.391093,0.864474
150,No log,1.902439,0.686869,0.376243,0.789011
200,No log,1.817956,0.710101,0.387429,0.759579
250,No log,1.808305,0.711111,0.388764,0.767088
300,No log,1.816494,0.710101,0.414389,0.771485
350,No log,1.763887,0.712121,0.384572,0.760644
400,No log,1.768374,0.716162,0.386096,0.756844
450,No log,1.778724,0.717172,0.409109,0.751113
500,1.781500,1.776859,0.716162,0.397052,0.757102


TrainOutput(global_step=556, training_loss=1.7350998610901318, metrics={'train_runtime': 246.7422, 'train_samples_per_second': 143.826, 'train_steps_per_second': 2.253, 'total_flos': 2881505447426688.0, 'train_loss': 1.7350998610901318, 'epoch': 4.0})

In [150]:
trainer.save_model("./bert-multitask-final")
tokenizer.save_pretrained("./bert-multitask-final")

('./bert-multitask-final/tokenizer_config.json',
 './bert-multitask-final/special_tokens_map.json',
 './bert-multitask-final/vocab.txt',
 './bert-multitask-final/added_tokens.json',
 './bert-multitask-final/tokenizer.json')

In [151]:
df_test = pd.read_csv("trac2_CONVT_dev.csv")
df_test["EmotionalPolarity"] = np.ceil(df_test['EmotionalPolarity'])

columnsTest = df_test["text"]


In [152]:

listSentenceTest = []


for column in columnsTest:
    listSentenceTest.append(column) # Use original text


In [153]:
batchTest = tokenizer(listSentenceTest, padding=True, truncation=True, return_tensors="pt")
print(batchTest["input_ids"])
print(batchTest["attention_mask"])

tensor([[ 101, 7592, 2129,  ...,    0,    0,    0],
        [ 101, 7592,  999,  ...,    0,    0,    0],
        [ 101, 1045, 1005,  ...,    0,    0,    0],
        ...,
        [ 101, 6343, 2323,  ...,    0,    0,    0],
        [ 101, 1045, 3246,  ...,    0,    0,    0],
        [ 101, 3398, 1010,  ...,    0,    0,    0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])


In [154]:
labels_class = torch.tensor(df_test["EmotionalPolarity"].values, dtype=torch.int64)
labels_intensity = torch.tensor(df_test["Emotion"].values, dtype=torch.float)
labels_empathy = torch.tensor(df_test["Empathy"].values, dtype=torch.float)

print(torch.unique(labels_class))

tensor([0, 1, 2])


In [155]:
from datasets import Dataset

val_dataset = Dataset.from_dict({
    "input_ids": batchTest["input_ids"],
    "attention_mask": batchTest["attention_mask"],
    "labels_class": labels_class,
    "labels_intensity": labels_intensity,
    "labels_empathy": labels_empathy
})

In [156]:
eval_results = trainer.evaluate(eval_dataset=val_dataset)
print(eval_results)

{'eval_loss': 1.7687057256698608, 'eval_accuracy': 0.7181818181818181, 'eval_mse_intensity': 0.3925041200925177, 'eval_mse_empathy': 0.7563991597328387, 'eval_runtime': 1.4923, 'eval_samples_per_second': 663.399, 'eval_steps_per_second': 10.722, 'epoch': 4.0}


In [163]:
sentence = "I really enjoyed the movie, it was amazing!"
inputs = tokenizer(sentence, return_tensors="pt").to(trainer.model.device)

# Safe call
inputs.pop("token_type_ids", None)

with torch.no_grad():
    outputs = trainer.model(**inputs)

print(outputs)



{'loss': None, 'logits_cls': tensor([[ 1.7547, -0.3790, -1.3138]], device='cuda:0'), 'reg1': tensor([2.3261], device='cuda:0'), 'reg2': tensor([1.5535], device='cuda:0')}


In [164]:
from google.colab import files
import shutil

folder_to_download = "bert-multitask-final"
output_zip_filename = "bert.zip"

shutil.make_archive(output_zip_filename.replace(".zip", ""), 'zip', folder_to_download)

files.download(output_zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>